# Does the linear regression make sense?

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## RegNetWeb

In [123]:
# import datasets
test = pd.read_csv('regnetworkweb_all8tf.csv')

# greater than 100
smaller = pd.read_pickle('test_data.pkl')

# greater than 50
data = pd.read_pickle('larger_data.pkl')


In [87]:
# capitalize first letter
test['regulator_symbol'] = [x.capitalize() for x in test['regulator_symbol']]
test['target_symbol'] = [x.capitalize() for x in test['target_symbol']]

In [124]:
data

predictors  target
Cdk4        Aars      -67.940078
            Abce1     -59.373681
            Adarb1     75.145700
            Adk        59.045460
            Agpat4   -134.814693
                         ...    
Tox         Ssrp1     -52.792687
            Suz12     -52.047600
            Tox      -159.187696
            Tspan3    -54.590385
            Uba2      -51.438388
Name: coefficients, Length: 714, dtype: float64

In [128]:
# to work with data
data = data.reset_index()

In [106]:
# Comparing Data from RegNetWeb

predictors = data['predictors'].drop_duplicates().tolist()

summary = {}

for x in predictors:
    t = test[test['regulator_symbol']==x]['target_symbol'].tolist()
    d = data[data['predictors']==x]['target'].tolist()
    #print(x)
    #print('This is test', t)
    #print('This is model', d)
    s = list(set(t) & set(d))
    summary[x]={'total': len(t), 'match': s}

Cdk4
This is test []
This is model ['Aars', 'Abce1', 'Adarb1', 'Adk', 'Agpat4', 'Agpat5', 'Anxa2', 'Apex1', 'Atad3a', 'Aunip', 'Aurka', 'Bard1', 'Baz1a', 'Brix1', 'Bzw2', 'Cbx5', 'Ccdc50', 'Cct8', 'Cd3e', 'Cd3g', 'Cdc6', 'Cdk1', 'Cdt1', 'Cenpi', 'Cenpp', 'Chchd4', 'Cmtm7', 'Cnbp', 'Dbf4', 'Desi2', 'Dimt1', 'Dnajb1', 'Dnajc2', 'Dtl', 'Dut', 'E2f1', 'Eed', 'Entpd1', 'Ezh2', 'Ftsj3', 'Glipr2', 'Gm26917', 'Gng2', 'Gspt1', 'Hif1a', 'Hnrnpdl', 'Hp1bp3', 'Hspa9', 'Icos', 'Impdh2', 'Irf2', 'Itga4', 'Itgb7', 'Kpna3', 'Lgals1', 'Lgals3', 'Lig1', 'Map1b', 'Mbnl1', 'Mcm4', 'Mpp6', 'Mrpl1', 'Naa50', 'Nebl', 'Nfkbia', 'Nop2', 'Npm1', 'Nsl1', 'Nucks1', 'Nufip1', 'Orc6', 'Pde10a', 'Pitpnc1', 'Pole', 'Prc1', 'Prim2', 'Prkch', 'Psmd7', 'Ranbp1', 'Rangap1', 'Rap1gap2', 'Rbbp8', 'Rcc2', 'Rcl1', 'Rev1', 'Rexo2', 'Rilpl2', 'Rpf2', 'Rpp30', 'Rrp1b', 'Rsbn1', 'Satb1', 'Sgk1', 'Skap1', 'Sla', 'Snrpd1', 'Spats1', 'Spire1', 'Srrm1', 'Stk17b', 'Stk24', 'Tagln2', 'Timm9', 'Tipin', 'Tmem11', 'Tmem71', 'Tnfrsf4', 'T

In [108]:
summary

{'Cdk4': {'total': 0, 'match': []},
 'Hif1a': {'total': 109, 'match': ['Pdgfb']},
 'Hmgb2': {'total': 0, 'match': []},
 'Klf2': {'total': 3, 'match': []},
 'Mcm3': {'total': 9, 'match': []},
 'Mcm5': {'total': 9, 'match': []},
 'Mcm6': {'total': 9, 'match': []},
 'Tox': {'total': 0, 'match': []}}

There are no matches. Out of 139 target genes found for ['Cdk4', 'Hif1a', 'Hmgb2', 'Klf2', 'Mcm3', 'Mcm5', 'Mcm6', 'Tox'] there where no matches. 

Looking at about 700 genes with mean/sd values > 50, there is one match, Pdgfb, with a ratio of -61.66.

In [110]:
data[data['target']=='Pdgfb']

,predictors,target,coefficients
183,Hif1a,Pdgfb,-61.660615
594,Mcm6,Pdgfb,-70.004292


check coefficients without dividing by the variation -> look at absolutely high variation coefficients
final list of avarage coefficients: divide all values by average coefficient -> higher and lower rank..
treat coefficients as score to calculate z-scores and get pvalues -> threshold can be significance. 

# GSEA

In [113]:
smaller = data.reset_index()

In [121]:
smaller['target'] = [x.upper() for x in smaller['target']]

In [122]:
smaller[['target', 'coefficients']].to_csv('greater50.csv')

# Over representation Analysis ORA